In [4]:

from config.database import collection, collection_weather, collection_searchTrack
from routes.root.root_class import Root_class, Fetching_Mechanism, Source_links_and_api, Root_source_links
from bson import ObjectId
import requests
import datetime as dt
import json
import pickle
from pymongo import UpdateOne


# get synopsis of the collection:
x = [i for i in collection_searchTrack.find({})]

# x = collection_searchTrack.delete_many({})
x

# [i for i in collection_searchTrack.find_one({})]

[]

In [13]:
x[0]
for data in x:

    update_query = {
        "$setOnInsert": {"email": data['email']},  # Only set email on document creation
        "$inc": {f"searches.{data['searchTerm']}": 1},  # Increment count for this search term
        "$set": {"lastUpdated": data['timestamp']}  # Update timestamp
    }
    # This single operation will:
    # 1. Create document if email doesn't exist
    # 2. Create searchTerm with count 1 if it doesn't exist
    # 3. Increment count if searchTerm exists
    result = collection_searchTrack.update_one(
        {"email": data['email']},
        update_query,
        upsert=True
    )

    print({"status": "success", "matched_count": result.matched_count, "modified_count": result.modified_count})

{'status': 'success', 'matched_count': 0, 'modified_count': 0}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 0, 'modified_count': 0}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_count': 1}
{'status': 'success', 'matched_count': 1, 'modified_cou

In [14]:

y = [i for i in collection_searchTrack.find({})]
y

[{'_id': ObjectId('67ab9648f07a4d921a95d37e'),
  'email': 'ujasvaghani@gmail.com',
  'lastUpdated': datetime.datetime(2025, 2, 11, 7, 27, 5, 798000),
  'searches': {'newa': 3,
   'jfk': 1,
   '4433': 1,
   '443': 1,
   'n': 2,
   'newar': 3,
   'newark': 2,
   'me': 1,
   'new': 1,
   'Newark ': 1,
   'Newark Liberty International Airport (EWR)': 1,
   'Newark Liberty International Airport (EWR) ': 1}},
 {'_id': ObjectId('67ab9648f07a4d921a95d37f'),
  'email': 'ismailsakhani879@gmail.com',
  'lastUpdated': datetime.datetime(2025, 2, 10, 14, 36, 0, 784000),
  'searches': {'new york': 1,
   'New York Stewart International Airport (SWF)': 1,
   '414': 7}}]